# Plot Output Structure Validation

This notebook validates that plots are saved in the correct directory structure.

## Overview

- Check plot directory structure
- Validate file naming conventions
- Verify all formats are generated (PNG, PDF, ROOT, TXT)
- Check region summary placement


In [ ]:
import sys
from pathlib import Path
from datetime import datetime

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))


In [ ]:
# Check plot output directory structure
plots_dir = project_root / "outputs" / "plots"

if plots_dir.exists():
    # Get all version directories
    version_dirs = [d for d in plots_dir.iterdir() if d.is_dir() and not d.name.startswith('.')]

    if version_dirs:
        # Sort by name (should be datetime format)
        version_dirs.sort(key=lambda x: x.name, reverse=True)
        latest_version = version_dirs[0]

        print(f"Found {len(version_dirs)} version directories")
        print(f"Latest version: {latest_version.name}\n")

        # Check structure
        print("=== Directory Structure ===")
        expected_dirs = ["png", "pdf", "root", "text"]
        for dir_name in expected_dirs:
            dir_path = latest_version / dir_name
            if dir_path.exists():
                print(f"✓ {dir_name}/ exists")
            else:
                print(f"✗ {dir_name}/ missing")

        # Check region summary
        summary_png = latest_version / "region_summary.png"
        summary_pdf = latest_version / "region_summary.pdf"
        if summary_png.exists() or summary_pdf.exists():
            print(f"\n✓ Region summary found in version directory")
        else:
            print(f"\n✗ Region summary missing from version directory")

        # Check category structure
        print("\n=== Category Structure ===")
        for category in ["1b", "2b"]:
            png_cat_dir = latest_version / "png" / category
            if png_cat_dir.exists():
                print(f"✓ {category}/ exists in png/")
                # List regions
                regions = [d.name for d in png_cat_dir.iterdir() if d.is_dir()]
                print(f"  Regions: {regions}")
            else:
                print(f"✗ {category}/ missing in png/")
    else:
        print("No version directories found")
else:
    print("Plots directory does not exist. Run plotting first.")


In [ ]:
# Validate file naming convention
if plots_dir.exists() and version_dirs:
    latest_version = version_dirs[0]
    png_dir = latest_version / "png"

    if png_dir.exists():
        print("=== File Naming Validation ===")

        # Get sample files from different categories/regions
        sample_files = []
        for category_dir in png_dir.iterdir():
            if category_dir.is_dir():
                for region_dir in category_dir.iterdir():
                    if region_dir.is_dir():
                        files = list(region_dir.glob("*.png"))
                        if files:
                            sample_files.extend(files[:3])

        print(f"\nSample files ({len(sample_files)}):")
        for f in sample_files[:10]:
            # Expected format: {category}_{region_dir}_{variable_name}.png
            name_parts = f.stem.split("_")
            print(f"  {f.name}")
            print(f"    Parts: {name_parts}")

            # Validate format
            if len(name_parts) >= 3:
                category_part = name_parts[0]
                region_part = "_".join(name_parts[1:-1])
                var_part = name_parts[-1]

                assert category_part in ["1b", "2b"], f"Invalid category: {category_part}"
                print(f"    ✓ Category: {category_part}, Region: {region_part}, Variable: {var_part}")
            else:
                print(f"    ⚠ Unexpected format")


In [ ]:
# Check all formats are generated
if plots_dir.exists() and version_dirs:
    latest_version = version_dirs[0]

    print("=== Format Generation Validation ===")

    formats = {
        "PNG": latest_version / "png",
        "PDF": latest_version / "pdf",
        "ROOT": latest_version / "root",
        "TXT": latest_version / "text",
    }

    for format_name, format_dir in formats.items():
        if format_dir.exists():
            # Count files
            if format_name == "ROOT":
                files = list(format_dir.glob("*.root"))
            else:
                files = list(format_dir.rglob(f"*.{format_name.lower()}"))

            print(f"{format_name}: {len(files)} files")

            if format_name == "ROOT":
                # ROOT files should be one per variable
                print(f"  Sample: {[f.name for f in files[:5]]}")
        else:
            print(f"{format_name}: directory missing")
